In [8]:
import pandas as pd
import json
import urllib
import requests as rq
import base64
from datetime import datetime

In [9]:
import config
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

https://stackoverflow.com/questions/40023931/how-to-get-real-estate-data-with-idealista-api

In [10]:
def get_oauth_token(): 
    url = "https://api.idealista.com/oauth/token"    
    apikey= API_KEY #sent by idealist
    secret= API_SECRET #sent by idealista
    apikey_secret = apikey + ':' + secret
    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1]
    headers = {'Authorization' : 'Basic ' + auth,'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) #,'scope':'read'
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token

In [11]:
def search_api(token, URL):  
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

In [12]:
# values: es, it, pt
country = 'es' 
# search language for summary. values: es, it, pt, en, ca
locale = 'es' 
language = 'es' 
# items per page: 50 as maximum allowed
max_items = '50'
# values: sale, rent
operation = 'sale'
# values: homes, offices, premises, garages, bedrooms
property_type = 'homes'
# allowed values by property type:
# garages: distance, price, street, photos, publicationDate, modificationDate (rent only), weigh, pricedown
# premises: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2 (rent only), weigh, pricedown
# offices: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2, weigh, pricedown
# homes: distance, price, street, photos, publicationDate, modificationDate, size, floor, rooms, ratioeurm2 (sólo alquiler), weigh, pricedown
# rooms: distance, price, street, photos, publicationDate, modificationDate, floor
order = 'priceDown' 
# geographic coordinates (WGS84) (latitude,longitude)
center = '43.3773,-8.39852' 
# distance to center, in metres (ratio)
distance = '5000'
# values: asc, desc
sort = 'desc'
# 
maxPrice = '300000'
# numPage = 1

In [13]:
df = pd.DataFrame()
for i in range(1,15):
    numPage = str(i)
    url = ('https://api.idealista.com/3.5/'+country+
            '/search?operation='+operation+
            '&locale='+locale+
            '&maxItems='+max_items+
            '&order='+order+
            '&center='+center+
            '&distance='+distance+
            '&maxPrice='+maxPrice+
            '&propertyType='+property_type+
            '&sort='+sort+ 
            '&numPage='+numPage+
            '&language='+language)
    result = search_api(get_oauth_token(), url)
    df_temp = pd.json_normalize(result, record_path = ["elementList"])
    df = pd.concat([df, df_temp])
now = datetime.now()
date_time = now.strftime("%m%d%Y_%H:%M")
df.to_csv('idealista_search' + str(date_time) + '.csv')

OSError: Cannot save file into a non-existent directory: 'idealista_search10/21'

In [14]:
print(len(df))
df

700


,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,detailedType.typology,suggestedTexts.subtitle,suggestedTexts.title,parkingSpace.hasParkingSpace,parkingSpace.isParkingSpaceIncludedInPrice,detailedType.subTypology,labels,neighborhood,parkingSpace.parkingSpacePrice,newDevelopmentFinished
0,97358818,https://img3.idealista.com/blur/WEB_LISTING/0/...,126855,27,3,104000.0,flat,sale,95.0,True,...,flat,"Agra del Orzán - Ventorrillo, A Coruña",Piso en Calle Páramo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97422652,https://img3.idealista.com/blur/WEB_LISTING/0/...,556710,26,3,104000.0,flat,sale,85.0,True,...,flat,"Agra del Orzán - Ventorrillo, A Coruña",Piso en Calle Páramo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97504416,https://img3.idealista.com/blur/WEB_LISTING/0/...,31747,29,5,212000.0,duplex,sale,137.0,True,...,flat,"Los Castros - Castrillón, A Coruña",Dúplex en Calle Cerca,True,True,duplex,NaN,NaN,NaN,NaN
3,96363274,https://img3.idealista.com/blur/WEB_LISTING/0/...,B197,32,18,195000.0,flat,sale,109.0,True,...,flat,"Los Castros - Castrillón, A Coruña",Piso en Avenida de Oza,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,98951857,https://img3.idealista.com/blur/WEB_LISTING/0/...,08629,16,2,80000.0,flat,sale,60.0,False,...,flat,"Monte Alto - Zalaeta - Atocha, A Coruña",Piso,NaN,NaN,NaN,"[{'name': 'apartamentoType', 'text': 'Apartame...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,98873667,https://img3.idealista.com/blur/WEB_LISTING/0/...,5415A,26,3,250000.0,flat,sale,65.0,True,...,flat,"Juan Flórez-San Pablo, A Coruña",Piso en Calle Ciudad de Lugo,NaN,NaN,NaN,NaN,Juan Flórez-San Pablo,NaN,NaN
46,95969022,https://img3.idealista.com/blur/WEB_LISTING/0/...,3421-07152,34,7,190000.0,flat,sale,101.0,True,...,flat,"Cuatro Caminos - Plaza de la Cubela, A Coruña",Piso en Benito Blanco Rajoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,96697517,https://img3.idealista.com/blur/WEB_LISTING/0/...,353493,25,bj,140000.0,flat,sale,65.0,False,...,flat,"Eirís, A Coruña",Piso en Calle Lázaro Cárdenas,True,True,NaN,NaN,NaN,NaN,NaN
48,98812050,https://img3.idealista.com/blur/WEB_LISTING/0/...,E11 BJºC,21,bj,162367.0,flat,sale,62.0,True,...,flat,"Eirís, A Coruña",Piso en Calle Julio Montero s/n,True,True,NaN,NaN,NaN,NaN,False
